{/* cspell:ignore ccsd, ffsim, pvdz, Qunova, HI-VQE, AVAS, Dooh, pyscf, nucmod, GTO's, lucj, pseudopotential, pseudopotentials, magmom, Jastrow, multireference */}

In [1]:
# This cell is hidden from users
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
instance = service.active_account()["instance"]
backend_name = service.least_busy(operational=True, min_num_qubits=16).name

# HI-VQE Chemistry - A Qiskit Function by Qunova Computing

<Admonition type="note">
  <LegacyContent>
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/qunova-chemistry)
</LegacyContent>
<CloudContent>
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/qunova-chemistry)
</CloudContent>

  Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan users. They are in preview release status and subject to change.
</Admonition>

## Overview

In quantum chemistry, the electronic structure problem focuses on finding the solutions to the electronic Schrödinger equation - the quantum wave functions describing the behavior of the system's electrons. These wave functions are vectors of complex amplitudes, with each amplitude corresponding to the contribution of a possible electron configuration.

The ground state is the lowest energy wave function of the system and has a special importance in the study of molecular systems. The most accurate approach for computing the ground state considers all possible electron configurations, but this becomes intractable for larger systems since the number of configurations grows exponentially with system size.

The Handover Iterative Variational Quantum Eigensolver (HI-VQE) is an innovative hybrid quantum-classical method for accurately estimating the ground state of molecular systems. It integrates quantum hardware with classical computing, using quantum processors to efficiently explore candidate electron configurations and calculating the resulting wave function on classical computers. By generating compact yet chemically accurate wave functions, HI-VQE enhances research and discovery in quantum chemistry and materials science.

![Image showing an overview of Qunova's HI-VQE algorithm](/images/guides/qunova-chemistry/overview.avif)

HI-VQE reduces the computational complexity of the electronic structure problem by efficiently estimating the ground state with high accuracy. It focuses on a carefully selected subset of the most relevant electron configurations, optimizing both accuracy and efficiency.

Combining the strengths of both classical and quantum computers, HI-VQE iteratively refines and improves the current estimate wave function. Its unique subspace construction techniques help make configuration selection more efficient, so that users have greater computational control and improved accuracy in quantum chemistry simulations.

If you would like to learn about the algorithm in more depth, you can [read the associated research paper](https://arxiv.org/abs/2503.06292).

## Description

The number of electron configurations for a molecular system grows exponentially with system size. However, for certain electronic states, such as the ground state, it is common that only a small fraction of configurations significantly contribute to the state’s energy. Selected configuration interaction (SCI) methods exploit this sparsity to reduce computational costs by identifying and focusing on the most relevant configurations. This subset of configurations is referred to as a subspace.

HI-VQE leverages the inherent efficiency of quantum computers for representing molecular systems to aid the subspace search. It integrates classical and quantum subroutines to solve the electronic structure problem with high accuracy. Unlike existing quantum SCI methods, HI-VQE combines variational training, iterative subspace construction, and pre-diagonalization configuration screening to enhance efficiency by reducing quantum measurements, iterations, and classical diagonalization costs. HI-VQE can therefore be applied to larger molecular systems which require more qubits, and reduces the cost to solve a problem of a given size to the same degree of accuracy.

![Image showing a detailed description of each step in Qunova's HI-VQE algorithm.](/images/guides/qunova-chemistry/description.avif)

To compute a system's ground state, HI-VQE first uses the classical chemistry package PySCF to generate a molecular representation from user-provided inputs, such as the molecular geometry and other molecular information. It then enters a hybrid quantum-classical optimization loop, iteratively refining a subspace to optimally represent the ground state while minimizing the number of configurations included. The loop continues until convergence criteria, such as subspace size or energy stability, are met, after which the computed ground state wave function and energy are output. These results can be used to construct accurate potential energy surfaces and perform further analysis of the system.

The optimization loop focuses on adjusting a quantum circuit’s parameters to generate a high-quality subspace. HI-VQE offers three quantum circuit options: [`excitation_preserving`](/api/qiskit/qiskit.circuit.library.excitation_preserving), [efficient_su2](/api/qiskit/qiskit.circuit.library.efficient_su2), and [LUCJ](https://qiskit-community.github.io/ffsim/dev/explanations/lucj.html#The-local-unitary-cluster-Jastrow-(LUCJ)-ansatz). The optimization is initialized close to the Hartree-Fock reference state due to its general suitability. The circuit is then executed on a quantum device and configurations are sampled from the resulting quantum state before being returned as binary strings. Due to quantum device noise, some sampled configurations may be physically invalid, failing to conserve electron number or spin. HI-VQE addresses this using the configuration recovery process from the [qiskit-addon-sqd](/guides/qiskit-addons-sqd#sample-based-quantum-diagonalization-sqd-overview) package, so that users can either correct invalid configurations or discard them.

The valid configurations then undergo an optional screening step to remove those predicted to contribute minimally. This reduces the dimension of the subspace, thereby lowering the cost of the diagonalization step. If the screening is enabled, then a preliminary subspace Hamiltonian is constructed from the valid configurations and a diagonalization is performed with very loose termination critieria. Though the accuracy of the resulting amplitudes for each configuration is low, it is effective for predicting which configurations to leave out of the subspace this iteration, and it is fast to compute.

The selected configurations are added to the subspace, and the system's Hamiltonian is projected into this subspace. The subspace updates iteratively, preserving the most relevant configurations across iterations. This approach contrasts with alternative methods because the quantum circuit does not need to approximate the full ground state at each step.

Next, the subspace Hamiltonian is classically diagonalized to obtain the lowest eigenvalue and its corresponding eigenvector, representing an approximation of the ground state and its energy. As the subspace quality improves through iterations, the computed ground state better approximates the true ground state. An additional screening step can be performed at this point to remove any configurations from the subspace that don't have a substantial contribution to the computed ground state. This step ensures that the subspace carried into the next iteration is as compact as possible. This is evaluated based on the amplitudes that are returned by the diagonalization, as these represent each configuration's importance contribution to the computed ground state.

A convergence check then determines whether further training would improve results. If so, then an optional classical expansion step is performed, the quantum circuit parameters are updated to further minimize the computed energy, and the process repeats. The classical expansion step generates additional configurations for the subspace, supplementing the configurations sampled from the quantum device. It first identifies the configuration with the largest amplitude in the diagonalization results, before generating new configurations with single and double excitations from the identified configuration. The desired number of these configurations are then added to the subspace.

Once it is determined that the iterations have converged, HI-VQE returns the computed ground state (in the form of the states in the subspace and their amplitudes in the ground state wave function), its energy, and an energy variance measure that gives an indication of whether the computed state forms an eigenstate of the system's Hamiltonian.

Users are able to decide the quantum circuit used and the number of shots taken for each quantum circuit, as well as control the subspace size or enable the classical generation of additional configurations to assist the quantum generated configurations. In this way users can tailor the behavior of HI-VQE to suit their desired applications.

## Get started

First, [request access to the function](https://forms.office.com/r/zN3hvMTqJ1).
<LegacyContent>
Then, authenticate using your [IBM Quantum&reg; API key](http://quantum.ibm.com/) and select the Qiskit Function as follows:
</LegacyContent>
<CloudContent>
Then, authenticate using your [IBM Quantum&reg; API key](http://quantum.cloud.ibm.com/) and select the Qiskit Function as follows:
</CloudContent>

In [2]:
import reprlib
from qiskit_ibm_catalog import QiskitFunctionsCatalog

catalog = QiskitFunctionsCatalog()
function = catalog.load("qunova/hivqe-chemistry")

## Inputs

See the following table for all input parameters the function accepts. The subsequent [Molecule options](#molecule-options) and [HI-VQE options](#hi-vqe-options) sections go into more details about those arguments.

<LegacyContent>
| Name                   | Type                                                           | Description                                                                                                                                                                                                                                                                                                 | Required | Default                                                                  | Example                                                                                   |
|------------------------|----------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------|--------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| geometry               | Union[List[List[Union[str, Tuple[float, float, float]]]], str] | This can be either a string or structured lists containing atom and coordinate pairs. If this is given as a string, it must be a molecule geometry in Cartesian Coordinate Format. If this is given as a list, it should be given as a list of lists that each contain an atom string and coordinate tuple. | Yes      | N/A                                                                      | `[['O', (0, 0, 0)], ['H', (0, 1, 0)], ['H', (0, 0, 1)]]` or `"O 0 0 0; H 0 1 0; H 0 0 1"` |
| backend\_name          | str                                                            | Name of the backend to make the query.                                                                                                                                                                                                                                                                      | Yes      | N/A                                                                      | `ibm_fez`                                                                                 |
| instance               | str                                                            | The instance to use in that format.                                                                                                                                                                                                                                                                | No       | One is randomly picked if your account has access to multiple instances. | `hub1/group1/project1`                                                                    |
| max\_states            | int                                                            | The maximum subspace dimension for the diagonalization. Fewer states will be used if the number is not a perfect square.                                                                                                                                                                                                                                                    | Yes      | N/A                                                                      | `100`                                                                                     |
| max\_expansion\_states | int                                                            | The maximum number of classically-generated CI states to be included in each iteration.                                                                                                                                                                                                                     | Yes      | N/A                                                                      | `10`                                                                                      |
| molecule_options                | dict                                                           | Options related to the molecule used as input to HI-VQE. See the [Molecule options](#molecule-options) section for more details.                                                                                                                                                                                                                                                | No       | See the [Molecule options](#molecule-options) section for details.                                 | `{"basis": "sto3g", "unit": "angstrom" }`                               |
| hivqe_options                | dict                                                           | Options controlling the behavior of the HI-VQE algorithm. See the [HI-VQE options](#hi-vqe-options) section for more details.                                                                                                                                                                                                                                                | No       | See the [HI-VQE options](#hi-vqe-options) section for details.                                 | `{"shots": 10_000, "max_iter": 10 }`                               |
</LegacyContent>
<CloudContent>
| Name                   | Type                                                           | Description                                                                                                                                                                                                                                                                                                 | Required | Default                                                                  | Example                                                                                   |
|------------------------|----------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------|--------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| geometry               | Union[List[List[Union[str, Tuple[float, float, float]]]], str] | This can be either a string or structured lists containing atom and coordinate pairs. If this is given as a string, it must be a molecule geometry in Cartesian Coordinate Format. If this is given as a list, it should be given as a list of lists that each contain an atom string and coordinate tuple. | Yes      | N/A                                                                      | `[['O', (0, 0, 0)], ['H', (0, 1, 0)], ['H', (0, 0, 1)]]` or `"O 0 0 0; H 0 1 0; H 0 0 1"` |
| backend\_name          | str                                                            | Name of the backend to make the query.                                                                                                                                                                                                                                                                      | Yes      | N/A                                                                      | `ibm_fez`                                                                                 |
| instance               | str                                                            | The instance to use in that format.                                                                                                                                                                                                                                                                | No       | One is randomly picked if your account has access to multiple instances. | `CRN`                                                                    |
| max\_states            | int                                                            | The maximum subspace dimension for the diagonalization. Fewer states will be used if the number is not a perfect square.                                                                                                                                                                                                                                                    | Yes      | N/A                                                                      | `100`                                                                                     |
| max\_expansion\_states | int                                                            | The maximum number of classically-generated CI states to be included in each iteration.                                                                                                                                                                                                                     | Yes      | N/A                                                                      | `10`                                                                                      |
| molecule_options                | dict                                                           | Options related to the molecule used as input to HI-VQE. See the [Molecule options](#molecule-options) section for more details.                                                                                                                                                                                                                                                | No       | See the [Molecule options](#molecule-options) section for details.                                 | `{"basis": "sto3g", "unit": "angstrom" }`                               |
| hivqe_options                | dict                                                           | Options controlling the behavior of the HI-VQE algorithm. See the [HI-VQE options](#hi-vqe-options) section for more details.                                                                                                                                                                                                                                                | No       | See the [HI-VQE options](#hi-vqe-options) section for details.                                 | `{"shots": 10_000, "max_iter": 10 }`                               |
</CloudContent>

### Molecule options

The following table details all keys and values that can be set in the `molecule_options` dictionary, as well as their data types and default values. All keys are optional.

| Key                               | Value type                          | Default Value                    | Valid range                                                                                                                                                    | Explanation                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|:----------------------------------|:-----------------------------------:|:--------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `"charge"`                        | `int`                               | `0`                              | Various                                                                                                                                                        | An integer specifying the total net charge of the molecular system. The default value is 0; however, it can be any integer.                                                                                                                                                                                                                                                                                                                                                                                                              |
| `"basis"`                         | `str`                               | `'sto-3g'`                       | Various                                                                                                                                                        | A string specifying the basis type; these are passed to pyscf. (eg: `"sto-3g"`, `"3-21g"`, `"6-31g"`, `"cc-pvdz"`)                                                                                                                                                                                                                                                                                                                                                                                                                      |
| `"active_orbitals"`               | `List[int]`                         | Every orbital index.             | The spatial orbital indices valid for the problem.                                                                                                             | A list of active orbital indices in the interval [0, n) where n is the number of qubits used in the problem. If this is specified, then the frozen_orbitals argument must also be specified.                                                                                                                                                                                                                                                                                                                                            |
| `"frozen_orbitals"`               | `List[int]`                         | No indices.                      | The spatial orbital indices valid for the problem, excluding active orbitals.                                                                                  | A list of frozen orbital indices in the same range as active_orbitals. If specified, then active_orbitals must also be specified. Note that only occupied orbitals should be frozen as the number of active electrons is reduced by 2 for every occupied orbital that is frozen.                                                                                                                                                                                                                                                        |
| `"orbital_coeffs"`                | `List[List[float]]`                 | Hartree-Fock molecular orbitals. | Various.                                                                                                                                                       | The coefficients for the spatial orbitals used in the calculation of the electronic repulsion integrals for the system. Some valid examples are Hartree-Fock molecular orbitals, natural orbitals, and AVAS orbitals.                                                                                                                                                                                                                                                                                                                   |
| `"symmetry"`                      | `Union[str, bool]`                  | `False`                          | `True` or `False`                                                                                                                                              | Used to invoke point group symmetry for the initial molecular calculations to construct the symmetry adapted orbital basis. These symmetry-adapted orbitals are used as basis functions for the following SCF calculations. The default value is False; if set to True, then it will be invoked and arbitrary point groups will automatically be detected and used. If a particular symmetry is assigned, for example, symmetry = “Dooh”, then an error will be raised if the molecular geometry is not subject to this required symmetry. |
| `"symmetry_subgroup"`             | `Optional[str]`                     | `None`                           | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Can be used to generate a subgroup of the detected symmetry. This has no effect when symmetry is specified using the symmetry keyword argument.                                                                                                                                                                                                                                                                                                                                                                                         |
| `"unit"`                          | `str`                               | `"angstrom"`                     | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Specifies the measurement unit to use for atomic coordinates and distances. The default is to use angstrom units.                                                                                                                                                                                                                                                                                                                                                                                                                       |
| `"nucmod"`                        | `Optional[Union[dict, str]]`        | `None`                           | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Specifies the nuclear model to be used. By default it uses the point nuclear model, and other values enable Gaussian nuclear model. If a function is given, it will be used with the Gaussian nuclear model to generate the nuclear charge distribution value 'zeta'.                                                                                                                                                                                                                                                                   |
| `"pseudo"`                        | `Optional[Union[dict, str]]`        | `None`                           | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Specifies the pseudopotential for the atoms in the molecule. This is None by default, indicating that no pseudopotentials are applied and all electrons are explicitly included in the calculations.                                                                                                                                                                                                                                                                                                                                                |
| `"cart"`                          | `bool`                              | `False`                          | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Specifies whether to use cartesian GTOs as the angular momentum basis functions in the calculation. The default value of False will use spherical GTOs.                                                                                                                                                                                                                                                                                                                                                                               |
| `"magmom"`                        | `Optional[List[Union[int, float]]]` | `None`                           | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Sets the colinear spin magnetic moment of each atom. By default, this is None and each atom is initialized with a spin of zero.                                                                                                                                                                                                                                                                                                                                                                                                         |
| `"avas_aolabels"`                 | `Optional[List[str]]`               | `None`                           | i.e. ["H 1s", "O 2p"] for H2O                                                                                                                                                             | This defines Atomic Orbital to be included in AVAS scheme. See [AVAS documentation](https://arxiv.org/abs/1701.07862) .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| `"avas_threshold"`                | `float`                             | `0.2`                            |  Between 0.0 and 2.0                                                                                                                                                      |  This specifies the cutoff value used to determine which Atomic Orbitals (AOs) are retained in the active space.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| `"noons_level"`                   | `Optional[str]`                     | `None`                           | `"mp2"` or `"ccsd"`                                                                                                                                            | This defines the theoretical approach for preparing natural orbitals and selecting active orbitals based on the Natural Orbital Occupation Numbers (NOONs) scheme. See [NOONs documentation](https://doi.org/10.1063/5.0042147). Both the active and frozen orbital indices must be provided to control the number of orbitals (and the number of qubits).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |

### HI-VQE options

The following table details all keys and values that can be set in the `hivqe_options` dictionary, as well as their data types and default values. All keys are optional.

| Key                               | Value type                          | Default Value                    | Valid range                                                                                                                                                    | Explanation                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|:----------------------------------|:-----------------------------------:|:--------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `"shots"`                         | `int`                               | `1_000`                          | Between 1 and 10 000.                                                                                                                                          | Number of shots to use on the quantum device per iteration.                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| `"max_iter"`                      | `int`                               | `25`                             | Between 1 and 50.                                                                                                                                              | The maximum number of iterations to run to optimize the ansatz. The algorithm may use fewer iterations if convergence is achieved early.                                                                                                                                                                                                                                                                                                                                                                                                 |
| `"initial_basis_states"`          | `List[str]`                         | The Hartree-Fock state.          | Binary strings with the number of bits matching the required number of qubits for the problem.                                                                 | Can be used to restart the algorithm with classical states from a previous result.                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| `"ansatz"`                        | `str`                               | `"epa"`                          | `"epa"`, `"hea"`, or `"lucj"`                                                                                                                                  | This specifies the quantum ansatz to optimize to generate new states. `"epa"` selects the excitation-preserving ansatz. `"hea"` selects the hardware-efficient ansatz. `"lucj"` selects the local unitary cluster Jastrow ansatz.                                                                                                                                                                                                                                                                                                       |
| `"convergence_count"`             | `int`                               | `3`                              | At least 2.                                                                                                                                                    | The number of iterations without significant change of the computed energy that should elapse before the algorithm is deemed to have converged.                                                                                                                                                                                                                                                                                                                                                                                         |
| `"convergence_abstol"`            | `float`                             | `1e-4`                           | More than 0 and at most 1.                                                                                                                                     | The magnitude of change in computed energy that is considered significant for the purposes of convergence checks.                                                                                                                                                                                                                                                                                                                                                                                                                       |
| `"reset_convergence_count"`       | `bool`                              | `True`                           | `True` or `False`                                                                                                                                              | If `True`, the `convergence_count` iterations must occur without an interrupting significant change to qualify as converging. If `False`, then the algorithm will stop after `convergence_count` if insignificant changes have occurred at any iterations during the optimization process.                                                                                                                                                                                                                                                 |
| `"configuration_recovery"`        | `bool`                              | `True`                           | `True` or `False`.                                                                                                                                             | Whether or not to use configuration recovery from the `qiskit-addon-sqd` package. If True, invalid states sampled from the quantum device are corrected classically. If False, they are discarded.                                                                                                                                                                                                                                                                                                                                      |
| `"ansatz_entanglement"`           | `str`                               | `"circular"`                     | Any one of `"linear"`, `"reverse_linear"`, `"pairwise"`, `"circular"`, `"full"`, or `"sca"`. If using the `"lucj"` ansatz, `"lucj_default"` is also an option. | This specifies the entanglement scheme that should be used within the quantum circuit, following common Qiskit and [ffsim conventions for the LUCJ ansatz](https://qiskit-community.github.io/ffsim/dev/how-to-guides/lucj.html#How-to-simulate-the-local-unitary-cluster-Jastrow-(LUCJ)-ansatz).                                                                                                                                                                                                                                       |
| `"ansatz_reps"`                   | `int`                               | `2`                              | Greater than 0.                                                                                                                                                | The number of repetitions of each layer in the quantum circuit.                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| `"amplitude_screening_tolerance"` | `Union[float,int]`                  | `0`                              | At least 0, and less than 1.                                                                                                                                   | The tolerance for deciding which states should be screened out of the subspace after diagonalization. It specifies the inclusion threshold for the subspace states based on their computed amplitudes.                                                                                                                                                                                                                                                                                                                                  |
| `"overlap_screening_tolerance"`   | `float`                             | `1e-2`                           | Between `1e-4` and `1e-1`, inclusive.                                                                                                                          | The tolerance for predicting which states should be screened out of the subspace prior to diagonalization. It controls the accuracy of the predicted amplitudes for each state, with a smaller value resulting in more accurate predictions.                                                                                                                                                                                                                                                                                            |

## Outputs

The function returns a dictionary with four keys and values. The keys and values are documented in the following table:

| Key             | Value Type    | Explanation                                                                                                               |
|:----------------|---------------|---------------------------------------------------------------------------------------------------------------------------|
| `"energy"`      | `float`       | The approximate ground state energy of the molecule.                                                                      |
| `"states"`      | `List[str]`   | The selected determinants that form the subspace used to solve for the energy. They are in alternating alpha-beta format. |
| `"eigenvector"` | `List[float]` | The eigenvector corresponding to the ground state of the subspace composed of `"states"`.                                 |
| `"energy_variance"` | `float` | The energy variance of the ground state of the subspace composed of `"states"`, giving an indication of the quality of the solution. This value is non-negative and a lower value means that the ground state of the subspace more closely approximates an eigenstate of the system's Hamiltonian. |
| `"energy_history"` | `List[float]` | The energies computed each iteration during the hybrid optimization process, in the same order that they were computed. Two energies are computed per iteration as part of the SPSA optimization process. |

## Example

The first example shows how to compute the ground state energy for an NH3 molecule using the HI-VQE algorithm.

#### Define the molecular geometry and options

The molecular geometry of NH3 is provided with Cartesian coordinates separated with ";" for each atom.

In [3]:
# Define the molecule geometry
geometry = """
N         -0.85188       -0.02741        0.03141;
H          0.16545        0.00593       -0.01648;
H         -1.16348       -0.39357       -0.86702;
H         -1.16348        0.94228        0.06281;
"""

Additional options can be defined and provided for molecular system in the following dictionary format.

In [4]:
# Configure some options for the job.
molecule_options = {"basis": "sto3g"}
hivqe_options = {"shots": 100, "max_iter": 20}

Execute the function with geometry and option inputs.

In [5]:
# Run HI-VQE
job = function.run(
    geometry=geometry,
    # `backend_name` is the name of a backend with at least 16 qubits, for example, "ibm_marrakesh".
    backend_name=backend_name,
    # `instance` is your authenticated instance string, for example, "ibm-q/open/main" for the open plan
    instance=instance,
    max_states=2000,
    max_expansion_states=10,
    molecule_options=molecule_options,
    hivqe_options=hivqe_options,
)

It is a good idea to print the Function job ID so that it can be provided in support requests if something goes wrong.

In [6]:
print("Job ID:", job.job_id)

Job ID: 128ee399-7cfc-4be2-91e9-c4abd22b97c7


This example then utilizes 16 qubits with 8 orbitals of sto3g basis for an NH3 molecule.

Check your Qiskit Function workload's [status](/guides/functions#check-job-status) or return [results](/guides/functions#retrieve-results) as follows:

In [7]:
print(job.status())

QUEUED


After the job is completed, the results can be obtained with `result()` instance.

In [8]:
result = job.result()

# Output can be long, so we display a shortened representation
shortened_result = reprlib.repr(result)
print(shortened_result)

{'eigenvector': [0.9824200343205695, 0.009520846167419264, 6.365368845740147e-08, 3.6832123006425615e-07, 0.0012869941182066654, 2.3403891050875465e-05, ...], 'energy': -55.521146537970466, 'energy_history': [-55.52091922342852, -55.52113695367561, -55.521146537970466, -55.52114653864798, -55.521146537970466, -55.521146537970466, ...], 'energy_variance': 9.788555455342562e-12, ...}


To access the ground state energy, use the "energy" key. The "eigenvector" key provides the CI coefficients with corresponding bitstring notation of electron configuration stored with "states" of the results.

In [10]:
fci_energy = -55.521148034704126  # the exact energy using FCI method
hivqe_energy = result["energy"]
print(
    f"|Exact Energy - HI-VQE Energy|: {abs(fci_energy - hivqe_energy) * 1000} mHa"
)
print(f"Sampled Number of States: {len(result['states'])}")

|Exact Energy - HI-VQE Energy|: 0.0014967336596782843 mHa
Sampled Number of States: 1936


Output:

|Exact Energy - HI-VQE Energy|: 0.077237504 mHa
Sampled Number of States: 1936

## Performance

This section shows the demonstrated benchmark calculations of HI-VQE with a 24-qubit case for Li2S, a 40-qubit case for an N2 molecule, and a 44-qubit case for an FeP-NO system.

#### Dissociation potential energy surface curve for an Li2S molecule with 24 qubits

The PES curve is shown with the FCI reference and initial guess from RHF, together with the energy error from the FCI reference.

![Image showing that HI-VQE produces solutions within chemical accuracy of a classical reference PES curve for the Li2S system.](/images/guides/qunova-chemistry/Li2S_PES.avif)

The calculations have been conducted with the following geometries and options.

In [11]:
# This cell is hidden from users
backend_name = service.least_busy(operational=True, min_num_qubits=38).name

In [12]:
# Define Li2S geometries
Li2S_geoms = {
    "Li2S_1.51": "S        -1.239044    0.671232   -0.030374;Li       -1.506327    0.432403   -1.498949;Li       -0.899996    0.973348    1.826768;",
    "Li2S_2.40": "S        -1.741432    0.680397    0.346702;Li       -0.529307    0.488006   -1.729343;Li       -1.284307    0.989409    2.177209;",
    "Li2S_3.80": "S        -2.707255    0.674298    0.909161;Li        0.079218    0.552012   -1.671656;Li       -0.927010    0.931502    1.557063;",
}

# Configure some options for the job.
molecule_options = {
    "basis": "sto3g",
}
hivqe_options = {
    "shots": 100,
    "max_iter": 20,
}

results = []
for geom in ["Li2S_1.51", "Li2S_2.40", "Li2S_3.80"]:
    # Run HI-VQE
    job = function.run(
        geometry=Li2S_geoms[geom],
        backend_name=backend_name,  # can use any device with at least 38 qubits
        instance=instance,
        max_states=2000,
        max_expansion_states=10,
        molecule_options=molecule_options,
        hivqe_options=hivqe_options,
    )
    results.append(job.result())

The red dots represent the HI-VQE calculation results for six different geometries, and three geometries corresponding to 1.51, 2.40, and 3.80 Angstrom are provided as input in the above cell.

#### Dissociation PES curve for an N2 molecule with 40 qubits

The nitrogen molecule has been identified as a multireference system with large correlation energy contributions beyond the Hartree-Fock state. We conducted a benchmark calculation for the N2 molecule with cc-pvdz basis, (20o,14e) using the homo-lumo active orbital selection. The complete active space (CAS) number to represent this problem is 6,009,350,400. It is not possible to obtain the eigenvalue problem solution (for energy and electronic structure) with this number of states using a powerful desktop (16cpu/64GB). With HI-VQE, users can efficiently search the subspace of CAS states to find chemically accurate results while saving computation resources significantly. The following plots show the PES curve of 40 qubits HI-VQE calculation of the N2 molecule dissociation.

![Image showing that HI-VQE produces solutions within chemical accuracy of a classical reference PES curve for the N2 system.](/images/guides/qunova-chemistry/N2_PES_40qubits.avif)

#### Dissociation PES curve for five-coordinated iron(II)-porphyrin with an NO system with 44 qubits

Another interesting chemical system is an iron(II)-porphyrin (FeP) complex with a coordinated nitric oxide (NO) ligand, which represents a biologically relevant metalloporphyrin system that plays crucial roles in various physiological processes. In this example, HI-VQE has been utilized to estimate the accurate potential energy surface curve of the intermolecular interaction between FeP and NO (ground state energy for differently separated geometries). The combined system has 450 orbitals and 202 electrons (450o,202e) with 6-31g(d) basis in total. The homo-lumo active orbital selection was utilized to calculate the smaller case from the real case with (22o,22e). From the following benchmark results, we were able to achieve the chemical accuracy (>&nbsp;1.6 mHa) with a state-of-the-art classical computer chemistry calculation of CASCI(DMRG) (22o,22e) reference.

![Image showing that HI-VQE produces solutions within chemical accuracy of a classical reference PES curve for the FeP-NO system.](/images/guides/qunova-chemistry/fepno_44qubits.avif)

## Benchmarks

- Exact matrix size is the number of determinants for exact solution, such as FCI and CASCI.
- HI-VQE calculation samples and calculates the subspace of it (as in, HI-VQE matrix size).
- Total time includes QPU runtime and Qiskit Function runs with CPU.
- Accuracy is estimated from the energy difference from exact solution.

|   Chemical system  | Number of qubits | Exact matrix size |  HI-VQE matrix size  | E(diff) from exact (mHa) | Number of iteration | Total time    | QPU runtime usage |
| ------------------ | ---------------- | ----------------- | ------------------- | ------------------------ | ------------------- | ------------- | ----------------- |
|  $NH_3$ (8o,10e)   |  16              |  3136             |  1936               |  0.08                    |  6                  | 37 s          | 34 s              |
|  $Li_2S$ (10o,10e) |  20              |  63504            |  3969               |  0.60                    |  5                  | 250 s         | 50 s              |
|  $NH_3$ (15o,10e)  |  30              |  9018009          |  49729              |  0.90                    |  5                  | 354 s         | 54 s              |
|  $N_2$ (16o,14e)   |  32              |  130873600        |  1798281            |  1.10                    |  9                  | 6531 s        | 121 s             |
|  $3H_2O$ (18o,24e) |  36              |  344622096        |  399424             |  0.90                    |  24                 | 5174 s        | 130 s             |
|  $N_2$ (20o,14e)   |  40              |  6009350400       |  9012004            |  1.20                    |  21                 | 46547 s       | 258 s             |

## Fetch error messages

If your workload fails, the status will be `ERROR` and calling `job.result()` will raise an exception:

In [13]:
job = function.run(
    geometry="invalid-geometry",  # This will cause an error
    backend_name=backend_name,
    instance=instance,
    max_states=2000,
    max_expansion_states=15,
    molecule_options=molecule_options,
    hivqe_options=hivqe_options,
)

job.result()

QiskitServerlessException: {"message": "An unexpected error occurred during job execution. Please make sure that your inputs are valid. If you are still experiencing problems, you can contact the Qunova Computing support service at qiskit.support@qunovacomputing.com and provide the Function job ID of this job for more assistance.", "status": "failure"}

In [14]:
job.status()

'ERROR'

## Get support

You can send an email to [qiskit.support@qunovacomputing.com](mailto:qiskit.support@qunovacomputing.com) for help with this function.

If you want help with troubleshooting a specific error, please provide the Function job ID of the job that encountered the error.

## Next steps

<Admonition type="tip" title="Recommendations">
- Read [the associated research paper](https://arxiv.org/abs/2503.06292).
- Request access to the function by filling in [this form](https://forms.office.com/r/zN3hvMTqJ1).
- Try the [Compute dissociation PES curve for FeP-NO with HI-VQE](https://learning.quantum.ibm.com/tutorial/qunova-hivqe) tutorial.
</Admonition>